# Preliminary

In [ ]:
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.cluster import KMeans
import numpy as np
import os
import gc
os.chdir('G:/dataset/open_data_train_valid1/open_data_train_valid/train')
file_1 = pd.read_csv(os.path.join(os.getcwd(),"train_1.txt"),chunksize=100,sep='\t')
file_1 = file_1.get_chunk()
columns = file_1.columns
data_type = file_1.dtypes.copy()
id_1 = data_type[data_type== 'int64'].index
id_2 = data_type[data_type== 'float64'].index
d = {}
for each in id_1:
    d[each] = np.int64
for each in id_2:
    d[each] = np.float64
    def mem_usage(pandas_obj):
        if isinstance(pandas_obj, pd.DataFrame):
            usage_b = pandas_obj.memory_usage(deep=True).sum()
        else:
            usage_b = pandas_obj.memory_usage(deep=True)
        usage_mb = usage_b / 1024 ** 2
        return '{:03.2f}MB'.format(usage_mb)

In [93]:
names = []
for i in range(2,6):
    names.append(os.path.join(os.getcwd(),'train_%d.txt' %i))
labelled_iter = pd.read_csv(os.path.join(os.getcwd(),"train_1.txt"),chunksize=1000,sep='\t',dtype =d)
total = labelled_iter.get_chunk()
for name in names:
    unlabelled_iter = pd.read_csv(name,chunksize=1000,sep='\t',dtype =d,names = columns)
    unlabelled = unlabelled_iter.get_chunk()
    total = pd.concat([total,unlabelled],axis=0,ignore_index = True)
    del unlabelled
    gc.collect()

In [94]:
label = total['label']

In [95]:
missing_value = (total.isnull().sum()/total.isnull().count()).sort_values(ascending = False)
total.drop(missing_value[missing_value>0.5].index,1,inplace = True)

In [96]:
table = total.iloc[label[label.notnull()].index.values]

In [98]:
positive = label[label == 1].index.values
negative = label[label == 0].index.values
positive_sample = table.iloc[positive,:]
negative_sample = table.iloc[negative,:]

# the code below is not needed until the next markdown

In [99]:
positive_simu = positive_sample.sample(frac= 0.5)#n = 1000
negative_simu = negative_sample.sample(frac= 0.5)
simu_data = pd.concat([positive_simu,negative_simu],axis = 0)
simu_data.fillna(0,inplace = True)

In [100]:
id_ = simu_data.pop('id')
loan_dt = simu_data.pop('loan_dt')
tag = simu_data.pop('tag')

In [111]:
numerical_feature = simu_data.select_dtypes(include = 'float64')
#numerical_feature.fillna(1,inplace = True)

In [113]:
#columns = list(numerical_feature.columns)
#for column in numerical_feature.columns:
#    if ((numerical_feature[column].value_counts() / numerical_feature[column].value_counts().sum()) > 0.8).any() == True:
#      #  numerical_feature.drop(column, 1,inplace = True)
#        columns.remove(column)

In [60]:
#column = numerical_feat.columns[(numerical_feat.max() != np.inf)&(numerical_feat.min() != -np.inf)]
#numerical_feat = numerical_feat[column]

In [122]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
numerical_feat = scaler.fit_transform(numerical_feature)

In [63]:
Y = np.vstack((np.ones((positive_simu.shape[0],1)),np.zeros((negative_simu.shape[0],1))))

In [126]:
w = np.dot(numerical_feat.T,Y)

In [127]:
dealt = 0.9

In [128]:
w = w.ravel()

In [138]:
ind = w.argsort()[-int(dealt*Y.shape[0]):]

In [140]:
len(ind)

900

In [141]:
ind

array([3749, 3642,  815,  676, 2365, 2485,  297, 2532, 4705, 3682, 2033,
       3316, 3181, 4212,  655, 2631, 4069, 1142,  299, 2912, 1922, 4634,
        436, 4426, 4277, 3171, 3510, 2096, 3452,  135, 2554, 3068, 1070,
       1590, 4739,  893, 1926, 1101, 1325, 2788,  391,  548,  526, 2426,
       3844, 4271, 1122,  683, 2541, 3545, 1588,  456, 4425, 3385,  636,
        760, 3652, 3627, 1821,  443, 1703, 3197, 2302, 1879, 3620, 2127,
       2002, 2670, 1644, 2097, 2910, 2129, 2911, 1355, 2402, 4272, 4679,
       3711, 1595, 2111,  164, 2549,  644, 1423, 4787, 4015, 1254,  872,
       4703,  624, 2954, 3020, 1132, 4435, 1180, 1484, 2251,  972, 4241,
       3161,  300, 1213, 3630, 3205, 1238, 1938,  127, 3983, 3296, 4245,
        518, 3016,   75, 1048, 3959, 3797, 4337, 3818, 4294,  732, 4478,
       3843, 4660, 1228, 1349,  805, 3808,  828, 2580,  412, 2134, 1917,
       1529, 3848, 4385,  771, 2762, 2971, 1160, 1503, 1781, 3228,  605,
        822,   53, 3249, 3559, 3607, 3725, 4109, 22

In [143]:
numerical_feature.columns[ind]

Index(['f5203', 'f5052', 'f1154', 'f957', 'f3290', 'f3461', 'f406', 'f3523',
       'f6537', 'f5108',
       ...
       'f1243', 'f1495', 'f3079', 'f2075', 'f231', 'f3822', 'f5581', 'f1469',
       'f5567', 'f5773'],
      dtype='object', length=900)

# The code below is useful (Mutiple time SIS)

In [205]:
from numpy.random import choice
i = 0
indices = []
while i<=50:
    f1 = choice(positive_sample.index.values,size =int(positive_sample.index.values.shape[0]) ,replace=False )
    f2 = choice(negative_sample.index.values,size =int(0.5*negative_sample.index.values.shape[0]) ,replace=False )
    positive_simu = positive_sample.loc[f1]#n = 1000
    negative_simu = negative_sample.loc[f2]
    simu_data = pd.concat([positive_simu,negative_simu],axis = 0)
    simu_data.fillna(0,inplace = True)
    numerical_feature = simu_data.select_dtypes(include = 'float64')
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    numerical_feat = scaler.fit_transform(numerical_feature)
    Y = np.vstack((np.ones((positive_simu.shape[0],1)),-np.ones((negative_simu.shape[0],1))))
    w = np.dot(numerical_feat.T,Y)
    dealt = 0.9
    w = w.ravel()
    ind_ = w.argsort()[-int(dealt*Y.shape[0]):]
    i += 1
    indices.append(ind_)
indx = set(indices[0])
for each in indices:
    indx = indx.intersection(set(each))
indx

{9,
 18,
 22,
 31,
 34,
 39,
 41,
 48,
 52,
 59,
 72,
 74,
 80,
 90,
 92,
 104,
 137,
 138,
 149,
 165,
 170,
 172,
 173,
 187,
 194,
 203,
 205,
 221,
 228,
 231,
 240,
 253,
 254,
 256,
 274,
 278,
 302,
 328,
 333,
 343,
 348,
 354,
 359,
 377,
 382,
 385,
 401,
 412,
 422,
 426,
 441,
 443,
 445,
 450,
 454,
 463,
 473,
 476,
 477,
 480,
 495,
 511,
 514,
 522,
 524,
 541,
 543,
 575,
 622,
 630,
 636,
 644,
 655,
 693,
 716,
 728,
 768,
 794,
 797,
 805,
 808,
 816,
 820,
 822,
 827,
 832,
 835,
 842,
 844,
 855,
 859,
 870,
 872,
 879,
 887,
 903,
 905,
 920,
 931,
 937,
 947,
 954,
 965,
 970,
 974,
 987,
 999,
 1000,
 1002,
 1010,
 1013,
 1028,
 1031,
 1036,
 1040,
 1044,
 1048,
 1050,
 1052,
 1053,
 1065,
 1069,
 1101,
 1119,
 1142,
 1150,
 1158,
 1159,
 1160,
 1164,
 1168,
 1180,
 1189,
 1210,
 1220,
 1225,
 1226,
 1232,
 1243,
 1250,
 1271,
 1273,
 1290,
 1295,
 1297,
 1302,
 1315,
 1322,
 1344,
 1346,
 1353,
 1361,
 1362,
 1367,
 1380,
 1399,
 1401,
 1402,
 1408,
 1410,
 14

In [206]:
len(indx)

508

In [207]:
cl = numerical_feature.columns[list(indx)]